In [45]:
### Import packages

In [46]:
import pandas as pd
from cassandra.cluster import Cluster

In [47]:
### Create a session connection to Cassandra cluster

In [48]:
clstr = Cluster()
session = clstr.connect()

In [49]:
### Use session to 'talk' to cassandra

In [50]:
session.execute("DROP KEYSPACE IF EXISTS midtermkeyspace")

In the above code we are verifying whether keyspace named  midtermkeyspace exists and if so we are dropping the keyspace

In [51]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


We are trying to print existing keyspaces using desc (describe) keyspaces command

In [52]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS midtermkeyspace WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

We are creating a keyspace named midtermkeyspace with a replication factor 3 (means there are 3 copies of data)

In [53]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

midtermkeyspace
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


If we try to print the keyspaces , we can see that the midtermkeyspace is added

Create a filesys table for for storing userid,date and filetype and file.Here the primary key is userID

In [54]:
session.execute("""
CREATE TABLE IF NOT EXISTS midtermkeyspace.testfile ( 
    userID INT, 
    date DATE, 
    filetype TEXT, 
    filename TEXT,
    PRIMARY KEY(userID,filename)
);
""")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Unknown type midtermkeyspace.file"

### Load data

In [ ]:
df = pd.read_csv('middata.csv')
df

,userID,date,filetype,filename
0,101,07-10-2017,'image','bd.img'
1,102,19-09-2018,'text','cassandra.txt'
2,103,'30-05-2019','video','mongodb.mp4'
3,104,'09-11-2013,'image','cloud.img'
4,105,'14-05-2009','text','impala.txt'


In [ ]:
for index, row in df.iterrows():
    print(f"userID = {row.userID}, date ={row.date}, filetype = {row.filetype}, filename = {row.filename}")

userID = 101, date =07-10-2017, filetype = 'image', filename = 'bd.img'
userID = 102, date =19-09-2018, filetype = 'text', filename = 'cassandra.txt'
userID = 103, date ='30-05-2019', filetype = 'video', filename = 'mongodb.mp4'
userID = 104, date ='09-11-2013, filetype = 'image', filename = 'cloud.img'
userID = 105, date ='14-05-2009', filetype = 'text', filename = 'impala.txt'


In [ ]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midtermkeyspace.testfile (userID, date, filetype, filename)     
        VALUES ({row.userID}, {row.date}, {row.filetype}, {row.filename});
        """
    )
    session.execute(f"""
        INSERT INTO midtermkeyspace.testfile (userID, date, filetype, filename)     
        VALUES ({row.userID}, {row.date}, {row.filetype}, {row.filename});
        """
    )


        INSERT INTO midtermkeyspace.testfile (userID, date, filetype, filename)     
        VALUES (101, 07-10-2017, 'image', 'bd.img');
        


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 3:23 no viable alternative at input '-10' (...filename)             VALUES (101, 07[-10]...)">

Printing various fields and their corresponding values

In [ ]:
with open ("impala.txt:","w") as fout:
    fout.write("This is a code to demonstrate how to add text to a file. ")

Opening the file impala.txt in write mode and adding some random text 

In [ ]:
with open ("impala.txt:","r") as fin:
    tmp=fin.read()
tmp

'This is a code to demonstrate how to add text to a file. '

Opening the file in read mode and storing it in a temporary variable and trying to print the variable.

In [ ]:
d_rows=session.execute(f"""
        DELETE FROM midtermkeyspace.testfile WHERE userID=105 AND filename='impala.txt';
        """)

InvalidRequest: Error from server: code=2200 [Invalid query] message="Non PRIMARY KEY columns found in where clause: filename "

Deleting the file named imapla.txt by providing both the primary key attributes userid and filename

In [ ]:
rows = session.execute("select userID,date,filetype,filename from midtermkeyspace.testfile where userID=104 and filename='cloud.img' ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")
    

Selecting a record based on the userID and filename

Conclusion

In this, I created a namespace and table with the fields userID,date,filetype and filename. Retrieved the data from csv file named middata.csv and added the rows to the table.Opened the file and added text to the file, deleted the file by providing primary keys and selected a record.